In [1]:
import keras
keras.__version__

'2.11.0'

## The data

The cat vs dog dataset we use is not a Keras package. It was posted on Kaggle.com as part of a Computer Vision problem in late 2013, when ConvNets were not yet so popular. 

The images are medium resolution JPGEs. It looks like this:

![cats_vs_dogs_samples](https://s3.amazonaws.com/book.keras.io/img/ch5/cats_vs_dogs_samples.jpg)

It's no surprise that the 2013 Kaggle cat vs dog competition was won by ConvNets. The best were able to achieve up to 95% accuracy. In our example we are still far from this accuracy, but during the Deep Learning course we have learned how to approach this value using different methods to improve the performance of neural networks. It should be noted that in this example we are training on approximately only 10% of the data that was used for the contest. 
After downloading the dataset and decompressing it, we are going to create a new dataset containing three subsets: a training set containing 1000 images of each class, a validation set with 500 images of each class, and finally a test set with 500 images of each class.

Here we have a few lines of code that make us this distribution automatically:




In [2]:
import os, shutil

In [6]:
# The path to the directory where the original
# dataset was uncompressed
original_dataset_dir = 'train'

# The directory where we will
# store our smaller dataset
base_dir = 'cats_and_dogs_small'
os.mkdir(base_dir)

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# Directory with our validation cat pictures
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# Directory with our validation dog pictures
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# Copy first 1000 cat images to train_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join('train\\cats', fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# Copy next 500 cat images to validation_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join('validation\\cats', fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 cat images to test_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join('test\\cats', fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy first 1000 dog images to train_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join('train\\dogs', fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to validation_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join('validation\\dogs', fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to test_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join('test\\dogs', fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

As a sanity check, let's count how many pictures we have in each training split (train/validation/test):

In [7]:
print('total training cat images:', len(os.listdir(train_cats_dir)))

total training cat images: 1000


In [8]:
print('total training dog images:', len(os.listdir(train_dogs_dir)))

total training dog images: 1000


In [9]:
print('total validation cat images:', len(os.listdir(validation_cats_dir)))

total validation cat images: 500


In [10]:
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

total validation dog images: 500


In [11]:
print('total test cat images:', len(os.listdir(test_cats_dir)))

total test cat images: 500


In [12]:
print('total test dog images:', len(os.listdir(test_dogs_dir)))

total test dog images: 500


So effectively we have 2000 training images, 1000 validation images and 1000 test images. In each of these subsets there are the same number of examples from each class: this is what is called a balanced binary classification system, which means that our classification accuracy will be an adequate metric of the success of our solution.

## Building our network


In [3]:
from keras import layers
from keras import models

model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128, (3,3), activation='relu'))  
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128, (3,3), activation='relu'))  
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

For the compilation step, we will be using `RMSprop`(lr=1e-4) as optimizer. As our network finished with a sigmoid, we are going to use binary cross entropy as loss function.

In [4]:
from keras import optimizers

model.compile(keras.optimizers.RMSprop() , loss=keras.losses.binary_crossentropy, metrics=['acc'])

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

validation_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory(
        'cats_and_dogs_small\\train',
        target_size=(150, 150), 
        batch_size=20,
        class_mode='binary') 
test_generator = test_datagen.flow_from_directory(
        'cats_and_dogs_small\\test',
        target_size=(150, 150), 
        batch_size=20,
        class_mode='binary') 
validation_generator = validation_datagen.flow_from_directory(
        'cats_and_dogs_small\\validation',
        target_size=(150, 150),  
        batch_size=20,
        class_mode='binary') 

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


Let's take a look at one of these generators: it takes us to a batch of 150x150 RGB images (dimensions `(20, 150, 150, 3)`) and binary tags (dimension `(20,)`). 20 is the number of examples in each batch (what we call the batch size). The generator generates these batches indefinitely: it runs a loop endlessly through all the images we have in the folder. That's why we have to type `break` to break the loop at some point.


In [65]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


Now let's make the fit. In this case, as what we have is a generator, we use fit_generator. We are going to run 30 epochs and use the validation dataset.

In [6]:
history = model.fit_generator(generator = train_generator, steps_per_epoch = 20, epochs=30)

C:\Users\nacho\AppData\Local\Temp\ipykernel_8324\3774117403.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator = train_generator, steps_per_epoch = 20, epochs=30)


Epoch 1/30
20/20 [==============================] - 40s 2s/step - loss: 0.8691 - acc: 0.4675
Epoch 2/30
20/20 [==============================] - 42s 2s/step - loss: 0.6947 - acc: 0.5050
Epoch 3/30
20/20 [==============================] - 40s 2s/step - loss: 0.7201 - acc: 0.5700
Epoch 4/30
20/20 [==============================] - 40s 2s/step - loss: 0.6939 - acc: 0.5375
Epoch 5/30
20/20 [==============================] - 35s 2s/step - loss: 0.6901 - acc: 0.5825
Epoch 6/30
20/20 [==============================] - 41s 2s/step - loss: 0.6903 - acc: 0.5900
Epoch 7/30
20/20 [==============================] - 43s 2s/step - loss: 0.6707 - acc: 0.6400
Epoch 8/30
20/20 [==============================] - 46s 2s/step - loss: 0.7383 - acc: 0.6225
Epoch 9/30
20/20 [==============================] - 39s 2s/step - loss: 0.6528 - acc: 0.6050
Epoch 10/30
20/20 [==============================] - 40s 2s/step - loss: 0.6319 - acc: 0.6375
Epoch 11/30
20/20 [==============================] - 42s 2s/step - lo

In [7]:
test_loss, test_acc = model.evaluate(test_generator)

50/50 [==============================] - 31s 610ms/step - loss: 0.6315 - acc: 0.7140


Saving the model after training

In [8]:
model.save('modelo_perrosGatos.h5')

Let's now load the saved model and evaluate it using the test dataset

In [9]:
import tensorflow as tf
modelo = tf.keras.models.load_model('modelo_perrosGatos.h5')
predicciones = modelo.predict(validation_generator)

50/50 [==============================] - 32s 636ms/step


Optimization of the network

Let's try to flip our images and retraining

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2, 
        horizontal_flip=True)


# Carga las imágenes de entrenamiento desde un directorio

train_generator = train_datagen.flow_from_directory(
        'cats_and_dogs_small\\train',
        target_size=(150, 150),  # Redimensiona las imágenes a 150x150 píxeles
        batch_size=32,
        class_mode='binary') 
test_generator = train_datagen.flow_from_directory(
        'cats_and_dogs_small\\test',
        target_size=(150, 150),  # Redimensiona las imágenes a 150x150 píxeles
        batch_size=32,
        class_mode='binary') 
validation_generator = train_datagen.flow_from_directory(
        'cats_and_dogs_small\\validation',
        target_size=(150, 150),  # Redimensiona las imágenes a 150x150 píxeles
        batch_size=32,
        class_mode='binary') 

history = model.fit_generator(generator = train_generator, steps_per_epoch = 20, epochs=30)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


C:\Users\nacho\AppData\Local\Temp\ipykernel_8324\4167916078.py:27: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator = train_generator, steps_per_epoch = 20, epochs=30)


Epoch 1/30
20/20 [==============================] - 61s 3s/step - loss: 0.5281 - acc: 0.7422
Epoch 2/30
20/20 [==============================] - 58s 3s/step - loss: 0.4986 - acc: 0.7564
Epoch 3/30
20/20 [==============================] - 54s 3s/step - loss: 0.5179 - acc: 0.7676
Epoch 4/30
20/20 [==============================] - 55s 3s/step - loss: 0.5025 - acc: 0.7750
Epoch 5/30
20/20 [==============================] - 55s 3s/step - loss: 0.5184 - acc: 0.7266
Epoch 6/30
20/20 [==============================] - 55s 3s/step - loss: 0.4905 - acc: 0.7812
Epoch 7/30
20/20 [==============================] - 54s 3s/step - loss: 0.5053 - acc: 0.7469
Epoch 8/30
20/20 [==============================] - 59s 3s/step - loss: 0.4808 - acc: 0.7922
Epoch 9/30
20/20 [==============================] - 57s 3s/step - loss: 0.4841 - acc: 0.7656
Epoch 10/30
20/20 [==============================] - 54s 3s/step - loss: 0.4367 - acc: 0.7953
Epoch 11/30
20/20 [==============================] - 55s 3s/step - lo

In [13]:
test_loss, test_acc = model.evaluate(test_generator)

32/32 [==============================] - 33s 1s/step - loss: 0.5340 - acc: 0.7480


FLiping the images, our accuracy changes from 0.714 to 0.748